In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## imports
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import time
import logging
import json
logging.basicConfig(filename='./logs/train-sum-qa-aug.log', level=logging.INFO)
from torch.utils.data import DataLoader

from sentence_transformers import SentenceTransformer, util, evaluation, losses, InputExample
from TypingDataset import TypingDataset, MSELoss

In [3]:
## data_paths for NLI

data_folder_path = '/drive02/novak/datasets/downloads/qa/'

data_filenames = [
    'gooaq_pairs.jsonl',
    'yahoo_answers_title_answer.jsonl',
    'stackexchange_duplicate_questions_title_title.jsonl',
    'eli5_question_answer.jsonl',
    'yahoo_answers_title_question.jsonl',
    'yahoo_answers_question_answer.jsonl',
    'wikihow.jsonl',
    'NQ-train_pairs.jsonl',
    'amazon-qa-train-pairs.jsonl',
    'squad_pairs.jsonl',
    'quora_duplicates.jsonl',
    'PAQ_pairs.jsonl',
]

def load_data(filename):
    
    ret = []
    with open(f'{data_folder_path}{filename}', 'r') as rf:
        
        # lines = rf.readlines()
        
        lines = [InputExample(texts=json.loads(line)) for line in tqdm(rf)]
        
        return lines
        
        # for line in tqdm(lines):
        #     line_data = InputExample(texts=json.loads(line))
        #     ret.append(line_data)
        
    # return ret
            


In [4]:
data = []
for dataset in data_filenames:
    
    data += load_data(dataset)

4660it [00:00, 33746.25it/s]

3012496it [00:15, 191360.28it/s]
1198260it [00:06, 194348.85it/s]
304525it [00:00, 338793.58it/s]
325475it [00:02, 114406.00it/s]
659896it [00:02, 286870.84it/s]
681164it [00:05, 128521.47it/s]
128543it [00:00, 322693.03it/s]
100231it [00:00, 207975.76it/s]
2448839it [00:10, 236256.41it/s]
87599it [00:00, 201247.54it/s]
103663it [00:03, 31918.90it/s]
21715049it [02:15, 209307.39it/s]

In [ ]:
print(len(data))
pair_data_loader = DataLoader(data, batch_size=300*4, shuffle=True)

73422132


In [ ]:
model_name = 'all-MiniLM-L12-v2'
print('djovak')
model = SentenceTransformer(model_name, device='cuda:0', use_parallel_gpu=True)



In [ ]:
from sentence_transformers import models
import torch
model_name = 'qa-yotta-L12-v1'
word_embedding_model = models.Transformer('microsoft/MiniLM-L12-H384-uncased', max_seq_length=512, tokenizer_args=tokenizer_args)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=384, activation_function=torch.nn.Tanh())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

In [ ]:
negative_loss = losses.MultipleNegativesRankingLoss(model=model, name='negative', similarity_fct=util.cos_sim, scale=80)

In [ ]:
# import torch


# with torch.no_grad():
#     torch.cuda.empty_cache()

In [ ]:
try:
    model.fit(train_objectives=[(pair_data_loader, negative_loss)], 
            epochs=10,
            use_amp=True,
            warmup_steps=300, 
            weight_decay = 0.001,
            evaluation_steps=100, 
            checkpoint_save_total_limit=3,
            checkpoint_save_steps=100,
            checkpoint_path = f'./typing-checkpoints/qa-summed-{model_name}/',
            output_path=f'./typing/qa-summed-{model_name}/')
finally:
    import torch
    with torch.no_grad():
        torch.cuda.empty_cache()

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/61186 [00:00<?, ?it/s]

OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/novak/.local/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/home/novak/.local/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/novak/.local/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/novak/.local/lib/python3.8/site-packages/torch/nn/modules/container.py", line 215, in forward
    input = module(input)
  File "/home/novak/.local/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/novak/.local/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_cal